# Atelier de Formation en Statistique Génétique et Épidémiologie Génétique STAGE-Québec
## Thème 2 - Phénotypes moléculaires en épidémiologie génétique

Par Marc-André Legault (Université de Montréal) et Qihuang Zhang (Université McGill)

**31 juillet - 1er août 2025**

### Introduction

Cet atelier offre une introduction aux études d'association du transcriptome imputé (TWAS). Nous démontrerons comment identifier les gènes du diabète de type 2 (DT2) en testant l'association entre l'expression génique prédite dans divers tissus et le risque de DT2.

Nous utiliserons des données réelles provenant d'une méta-analyse à grande échelle impliquant 180 834 cas de DT2 et plus d'un million de témoins de [Mahajan _et al._ (2022)](https://www.nature.com/articles/s41588-022-01058-3). Ce sont des statistiques sommaires de haute qualité qui représentent les standards actuels en épidémiologie génétique.

Cet atelier couvrira deux approches TWAS établies :
- **[S-PrediXcan](https://www.nature.com/articles/s41467-018-03621-1)**
- **[TWAS FUSION](https://www.nature.com/articles/ng.3506)**

Les statistiques sommaires GWAS ont été harmonisées pour correspondre aux modèles de prédiction d'expression génique. Nous avons créé un ensemble ciblé de tissus provenant de GTEx pour équilibrer l'exhaustivité et l'efficacité computationnelle.

### Structure de l'atelier

L'atelier est organisé en trois sections interconnectées :

1. **S-PrediXcan** (ce carnet) - Fondements du TWAS basé sur les statistiques sommaires utilisant S-PrediXcan.
2. **TWAS / FUSION** [(lien)](2a-FUSION.ipynb) - Un deuxième logiciel pour effectuer des TWAS.
3. **Comparaison des méthodes** [(lien)](3a-compare_TWAS_models.ipynb) - Analyse comparative des résultats.

**Navigation :** Utilisez l'explorateur de fichiers dans le panneau de gauche pour explorer tous les matériaux de l'atelier et explorer les différents tutoriels.

<div class="alert alert-warning">
    <p>
    <strong>Important :</strong> Le dossier <strong>local</strong> est partagé avec votre ordinateur—tout le reste disparaîtra quand vous fermerez le serveur. Sauvegardez votre travail et vos résultats importants dans ce dossier.
    </p>
    <p>
    Le dossier « local » dans cet environnement se trouve sur votre ordinateur, dans votre répertoire personnel :<br/>
    • Windows : <code>C:\Users\[votre nom d'utilisateur]\STAGE2025_workshop_theme2</code><br/>
    • macOS/Linux : <code>/Users/[votre nom d'utilisateur]/STAGE2025_workshop_theme2</code>
    </p>
</div>

Nous commencerons par explorer S-PrediXcan et l'appliquer à nos données GWAS préparées pour investiguer l'architecture génétique du diabète de type 2.

### Exploration des statistiques sommaires GWAS

Avant d'exécuter S-PrediXcan, nous examinerons nos données d'entrée. Comprendre la structure et le contenu des statistiques sommaires GWAS est essentiel pour toute analyse TWAS.

Nos statistiques sommaires GWAS ont été harmonisées avec les modèles [PredictDB](https://predictdb.hakyimlab.org/)—les modèles standards de prédiction d'expression génique qui accompagnent S-PrediXcan. Cette harmonisation assure que les noms de variants, le codage des allèles et les coordonnées génomiques sont alignés entre nos données GWAS et les modèles de prédiction.

**Note :** Cette étape d'harmonisation est critique car les incohérences dans la nomenclature des variants ou le codage des allèles peuvent mener à des résultats incorrects ou à des associations manquées.

Examinons la structure des données, qui suit un format cohérent dans tous les tissus :

In [1]:
filename="/workshop/data/PrediXcan/gwas_harmonized/harmonized_en_Whole_Blood.tsv.gz"
echo "Examen du fichier $filename"
echo "Numéros de colonnes et titres :"
echo "$(zcat $filename | head -n 1 | tr '\t' '\n' | nl)"
echo
echo "Extrait du fichier :"
zcat $filename | head -n 10 | cut -f 1-6,9

Examen du fichier /workshop/data/PrediXcan/gwas_harmonized/harmonized_en_Whole_Blood.tsv.gz
Numéros de colonnes et titres :
     1	chromosome
     2	base_pair_location
     3	effect_allele
     4	other_allele
     5	beta
     6	standard_error
     7	effect_allele_frequency
     8	neg_log_10_p_value
     9	rsid_x
    10	variant_id
    11	predictdb_id
    12	rsid_y

Extrait du fichier :
chromosome	base_pair_location	effect_allele	other_allele	beta	standard_error	rsid_x
chr1	785910	C	G	-0.0176	0.0262	rs12565286
chr1	817186	A	G	0.0048	0.0098	rs3094315
chr1	817341	G	A	0.0045	0.0097	rs3131972
chr1	825532	T	C	0.0069	0.0099	rs1048488
chr1	825767	C	T	0.0066	0.0098	rs3115850
chr1	833068	A	G	0.0	0.0117	rs12562034
chr1	841742	T	A	0.0078	0.0101	rs2980319
chr1	843942	G	A	-0.007	0.0101	rs4040617
chr1	849670	A	G	0.0063	0.0098	rs2905062


<div class="alert alert-info">
    <strong>Exercice :</strong> Prenez un moment pour interpréter les colonnes affichées. Que représente chaque colonne, et pourquoi chacune pourrait-elle être importante pour une analyse TWAS ?
</div>

Entrez votre réponse ici

### Modèles de prédiction d'expression génique : La fondation du TWAS

Nous examinerons maintenant comment l'expression génique est prédite à partir des variants génétiques. Les modèles de prédiction d'expression génique sont la pierre angulaire de toute analyse TWAS. Examinons les modèles pour le tissu « Sang total » (_Whole blood_) de GTEx comme exemple.

**Définition :** Ces modèles sont des fonctions mathématiques qui prédisent les niveaux d'expression génique basés sur les données génotypiques. Ils représentent la relation : « Étant donné le génotype d'une personne à des SNP spécifiques, quelle serait vraisemblablement son expression génique dans ce tissu ? » Ces modèles ont été entraînés en utilisant des données du consortium GTEx, où les chercheurs ont mesuré à la fois les génotypes et l'expression génique chez les mêmes individus.

PredictDB distribue ces modèles comme bases de données SQLite. Chaque base de données contient deux tables essentielles qui fonctionnent ensemble :

In [2]:
# Schéma pour la table "extra" :
sqlite3 /workshop/data/PrediXcan/predictdb/en_Whole_Blood.db ".schema extra"

CREATE TABLE `extra` (
  `gene` TEXT,
  `genename` TEXT,
  `gene_type` TEXT,
  `alpha` REAL,
  `n_snps_in_window` INTEGER,
  `n.snps.in.model` INTEGER,
  `test_R2_avg` REAL,
  `test_R2_sd` REAL,
  `cv_R2_avg` REAL,
  `cv_R2_sd` REAL,
  `in_sample_R2` REAL,
  `nested_cv_fisher_pval` REAL,
  `nested_cv_converged` INTEGER,
  `rho_avg` REAL,
  `rho_se` REAL,
  `rho_zscore` REAL,
  `pred.perf.R2` REAL,
  `pred.perf.pval` REAL,
  `pred.perf.qval` REAL,
  `phi` REAL
);
CREATE INDEX metab_model_summary ON extra (gene);


La table `extra` contient des informations de contrôle qualité du modèle. Pour chaque gène, elle inclut :
- **Identifiants de gènes** (ID Ensembl et nom du gène)
- **Composition du modèle** (combien de SNP sont inclus)
- **Métriques de performance** (à quel point le modèle prédit bien l'expression)

Ces métadonnées sont essentielles pour interpréter les résultats TWAS de manière appropriée.

<div class="alert alert-info">
    <strong>Question :</strong> Pourquoi la colonne <code>pred.perf.pval</code> est-elle importante ? Que nous dit-elle sur l'adéquation d'un gène pour l'analyse TWAS ? Considérez ce qui arrive si nous ne pouvons pas prédire de manière fiable l'expression d'un gène à partir des données génotypiques.
</div>

Entrez votre réponse ici

La table `weights` contient les paramètres de prédiction actuels—les composantes mathématiques de chaque modèle d'expression génique. Pour chaque gène (identifié par son ID Ensembl), cette table liste :
- **Quels variations génétiques** (eQTL) influencent l'expression
- **La taille de l'effet** de chaque variation (le poids de prédiction)

Examinons ceci avec un exemple. Nous regarderons le modèle pour **ENSG00000157823.16**, qui code pour AP3S2 (Adaptor Related Protein Complex 3 Subunit Sigma 2). Pour nous concentrer sur les contributeurs les plus importants, nous trierons les variants par leur poids absolu et montrerons les 10 premiers :

In [3]:
sqlite3 /workshop/data/PrediXcan/predictdb/en_Whole_Blood.db <<EOF
.mode columns
.header on
select * from weights
  where gene='ENSG00000157823.16'
  order by abs(weight) desc limit 10;
EOF

gene                rsid        varID                   ref_allele  eff_allele  weight             
------------------  ----------  ----------------------  ----------  ----------  -------------------
ENSG00000157823.16  rs4363847   chr15_90547112_T_C_b38  T           C           0.128329668848438  
ENSG00000157823.16  rs12898828  chr15_89817721_A_T_b38  A           T           -0.109067828981916 
ENSG00000157823.16  rs11073814  chr15_88835653_A_T_b38  A           T           0.0524825659916266 
ENSG00000157823.16  rs3853641   chr15_89900547_G_C_b38  G           C           -0.0467714907373544
ENSG00000157823.16  rs16943673  chr15_89864201_T_C_b38  T           C           -0.0326710086433488
ENSG00000157823.16  rs10520684  chr15_89881770_C_A_b38  C           A           -0.0295110487238457
ENSG00000157823.16  rs1879530   chr15_88902553_G_A_b38  G           A           0.0263159897074107 
ENSG00000157823.16  rs2165069   chr15_89863887_C_T_b38  C           T           -0.0251527944185157


<div class="alert alert-info">
    <strong>Interprétation des poids :</strong> Remarquez que certains poids sont positifs tandis que d'autres sont négatifs. Que cela nous dit-il sur la relation entre ces variants et l'expression génique ? Quelles méthodes statistiques pourraient être utilisées pour estimer ces poids de prédiction ?
</div>

Entrez votre réponse ici

### Exécution de votre première analyse TWAS avec S-PrediXcan

Nous sommes maintenant prêts à effectuer notre analyse TWAS en utilisant S-PrediXcan. Cette analyse combinera nos statistiques sommaires GWAS avec les modèles de prédiction d'expression génique pour identifier les gènes dont les niveaux d'expression prédits sont associés au risque de diabète de type 2.

**Note :** L'analyse ci-dessous peut prendre plusieurs minutes à compléter. Si elle prend trop de temps, vous pouvez l'arrêter et utiliser les fichiers de sauvegarde pré-calculés que nous avons fournis.

### Comprendre la commande S-PrediXcan

Chaque paramètre d'entrée sert un but spécifique :

**Fichiers d'entrée principaux :**
- **`model_db_path`** : La base de données SQLite contenant les poids de prédiction d'expression génique
- **`covariance`** : Estimations du déséquilibre de liaison (LD) entre les SNP—essentiel pour tenir compte de la corrélation entre variants voisins
- **`gwas_file`** : Nos statistiques sommaires GWAS harmonisées

**Paramètres de mappage de colonnes :**
Les paramètres `*_column` disent à S-PrediXcan comment interpréter le format de notre fichier GWAS. Cette flexibilité permet au logiciel de fonctionner avec des données de différentes sources et formats.

**Paramètres de calibration polygénique :**
- **Héritabilité SNP (`gwas_h2 = 0.15`)** : La proportion de variance du trait expliquée par tous les SNP
- **Taille d'échantillon (`gwas_N = 1000000`)** : La taille d'échantillon effective de notre GWAS

**Justification :** Les traits polygéniques comme le DT2 ont des milliers de variants causaux avec de petits effets distribués à travers le génome. Plusieurs de ces variants sont en déséquilibre de liaison avec nos eQTL, créant potentiellement des associations fallacieuses. En fournissant des estimations d'héritabilité et de taille d'échantillon, nous permettons à la méthode de recalibration de S-PrediXcan ([Liang _et al._ 2024](https://www.biorxiv.org/content/10.1101/2023.10.17.562831v2)) de tenir compte de ce « contexte polygénique » et de fournir des résultats plus précis.

L'analyse sera exécutée sur sept tissus sélectionnés :

In [5]:
tissues="Adipose_Subcutaneous Artery_Coronary Brain_Cortex Liver Muscle_Skeletal Pancreas Whole_Blood"

for tissue in $tissues; do
    echo
    echo
    echo "Exécution de S-PrediXcan pour le tissu ${tissue}"
    echo
    SPrediXcan.py \
        --model_db_path /workshop/data/PrediXcan/predictdb/en_${tissue}.db \
        --covariance /workshop/data/PrediXcan/predictdb/en_${tissue}.txt.gz \
        --gwas_file /workshop/data/PrediXcan/gwas_harmonized/harmonized_en_${tissue}.tsv.gz \
        --snp_column rsid_x \
        --effect_allele_column effect_allele \
        --non_effect_allele_column other_allele \
        --output_file /workshop/local/results/twas_en_${tissue}.csv \
        --beta_column beta \
        --se_column standard_error \
        --gwas_h2 0.15 \
        --gwas_N 1000000
    
    # Commentez cette ligne pour exécuter tous les tissus.
    break
done



Exécution de S-PrediXcan pour le tissu Adipose_Subcutaneous

INFO - Processing GWAS command line parameters
INFO - Building beta for /workshop/data/PrediXcan/gwas_harmonized/harmonized_en_Adipose_Subcutaneous.tsv.gz and /workshop/data/PrediXcan/predictdb/en_Adipose_Subcutaneous.db
INFO - Reading input gwas with special handling: /workshop/data/PrediXcan/gwas_harmonized/harmonized_en_Adipose_Subcutaneous.tsv.gz
INFO - Processing input gwas
INFO - Calculating zscore from se and beta
INFO - Aligning GWAS to models
INFO - Trimming output
INFO - Successfully parsed input gwas in 2.3616790429950925 seconds
INFO - Started metaxcan process
INFO - Loading model from: /workshop/data/PrediXcan/predictdb/en_Adipose_Subcutaneous.db
INFO - Loading covariance data from: /workshop/data/PrediXcan/predictdb/en_Adipose_Subcutaneous.txt.gz
INFO - Processing loaded gwas
INFO - Started metaxcan association
INFO - 10 % of model's snps found so far in the gwas study
INFO - 20 % of model's snps found so far 

### Analyse complétée

Vous avez exécuté avec succès S-PrediXcan sur plusieurs tissus et généré des résultats TWAS complets pour le DT2. Chaque fichier de sortie contient des statistiques d'association au niveau génique qui indiquent quels gènes ont des niveaux d'expression prédits significativement associés au risque de DT2 dans chaque tissu.

<div class="alert alert-info">
    <strong>Examinez vos résultats :</strong> Ouvrez un des fichiers de résultats dans le répertoire <code>results</code>. Cherchez des colonnes comme les noms de gènes, les valeurs p d'association et les tailles d'effet. Quels gènes montrent les associations les plus fortes ? En reconnaissez-vous comme gènes connus du diabète ?
</div>

Entrez votre réponse ici

### Prochaines étapes

Vous avez établi une fondation en méthodologie TWAS. La prochaine phase implique **l'interprétation** de ces résultats. Dans notre [prochain carnet](1b-S-PrediXcan-interpretation.ipynb), nous utiliserons l'environnement de programmation R pour :

- **Appliquer** les corrections pour tests multiples de manière appropriée
- **Visualiser** les patrons d'association à travers les tissus
- **Identifier** les gènes candidats les plus prometteurs
- **Comprendre** la signification biologique de nos découvertes